### Importing libraries

In [1]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from keras.callbacks import TensorBoard, ModelCheckpoint
from keras.optimizers import Adam
from keras.utils import plot_model
import time
import pandas as pd
import seaborn as sn
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import pickle
import json

# For using core package that located in the two upper folder.
import os,sys
sys.path.append('../../')

from core.json_importer import parse_json_file, parse_all_files
from core.filters import remove_both_standing_frames, remove_same_consecutive_actions, remove_recov_frames, remove_unchange_hp
from core.actions import one_hot_encode, one_hot_encode_hpdif, decode
from core.preproccessing import Normalizer
from core.helpers import write_file
from core.visualizer import plot_confusion_matrix
from util.AIConnection.predictor import Predictor

Using TensorFlow backend.


In [2]:
# Create out file folder if not exists
OUT_FOLDER = 'generated_files'
if not os.path.exists(OUT_FOLDER):
    os.makedirs(OUT_FOLDER)

In [3]:
# Open files
training_data = parse_all_files("data/trainbcp15/")
testing_data = parse_all_files("data/trainbcp5/")

In [4]:

# Apply filters for training data
tr_deleted = training_data.filter(remove_unchange_hp)

# Apply filters for training data
te_deleted = testing_data.filter(remove_unchange_hp)


print('There are {} train data.({} deleted.)'.format(len(training_data), tr_deleted))
print('There are {} test data.({} deleted.)'.format(len(testing_data), te_deleted))


There are 6116 train data.(155884 deleted.)
There are 1948 test data.(52052 deleted.)


#### Encoding and normalization

In [5]:
print(training_data)
 

There are 45 rounds and 6116 frames.


In [6]:
# hp values are -5 -4 -3 -2 -1 0 1 2 3 4 5
# and they are one hot encoding index 0 to index 10
# so, when we argmax to predicted result, if it is 0, it means our bot takes a lot of damage
#                                         if it is 10, it means our bot gives a lot of damage 

In [7]:
def normalize_and_save(data, file_name):
    p1_hp_normalizer = Normalizer()
    p1_normalized_hp = p1_hp_normalizer.normalize(data)
    p1_hp_normalizer.save(OUT_FOLDER + '/' + file_name)
    return p1_normalized_hp

def process_data(game_data_obj):
    ## Pre process data
    processed_data = []
    frames = [frame for frame in game_data_obj]
    actions = []
    p1x = []
    p1y = []
    p2x = []
    p2y = []
    hpdifs = []

    for i in range(0,len(frames)-1,2):
        actions.append(frames[i]['P1-action'])
        p1x.append(frames[i]['P1-x'])
        p1y.append(frames[i]['P1-y'])
        p2x.append(frames[i]['P2-x'])
        p2y.append(frames[i]['P2-y'])
        after = (frames[i+1]['P2-hp'] - frames[i+1]['P1-hp'])
        before = (frames[i]['P2-hp'] - frames[i]['P1-hp'])
        last = int(after - before) // 10
        if last < -5:
            last = -5
        if last > 5:
            last = 5
#       print("ust: " + str(ust) + ", alt: " + str(alt) + ", res: " + str(a))
        hpdifs.append(last)
    
    # Create one hot encoding for actions (For input and labels)
    p1_one_hot_encoded_actions = one_hot_encode(actions)
    labels = one_hot_encode_hpdif(hpdifs)

    # Normalize uncategorized features
    normalized_xp1_distance = normalize_and_save(p1x, "xp1_norm.save")
    normalized_xp2_distance = normalize_and_save(p2x, "xp2_norm.save")
    normalized_yp1_distance = normalize_and_save(p1y, "yp1_norm.save")
    normalized_yp2_distance = normalize_and_save(p2y, "yp2_norm.save")

    for index in range(len(p1_one_hot_encoded_actions)):    
        processed_row = []
        processed_row.extend(p1_one_hot_encoded_actions[index])
        processed_row.extend(normalized_xp1_distance[index])
        processed_row.extend(normalized_xp2_distance[index])
        processed_row.extend(normalized_yp1_distance[index])
        processed_row.extend(normalized_yp2_distance[index])
        processed_data.append(processed_row)
    processed_data = np.array(processed_data)
    labels = np.array(labels)
    
    return processed_data, labels

tr_data, tr_labels = process_data(training_data)
te_data, te_labels = process_data(testing_data)

### Example data

In [8]:
EXAMPLE_ROW = 10
print(tr_labels[6])
print("There are %d frames in dataset." % len(tr_data))
print("After pre processing the shape of our dataset is %s" % str(tr_data.shape))
print("\nOne example in index %d." % EXAMPLE_ROW)

row = tr_data[EXAMPLE_ROW]
print("\tProcessed Frame:" )
print("\t\tP1 Action(one-hot) Shape:\t[1, %d]" % 54)
print("\t\tP1 Hp(norm.):\t\t\t%s" % row[55])
print("\t\tP2 Hp(norm.):\t\t\t%s" % row[56])
print("\t\tX Dist(norm.):\t\t\t%s" % row[57])
print("\t\tY Dist(norm.):\t\t\t%s" % row[58])
print("\t\tLabel Shape:\t\t\t[1, %d]" % len(tr_labels[0]))

[0 0 0 0 0 0 1 0 0 0 0]
There are 3058 frames in dataset.
After pre processing the shape of our dataset is (3058, 59)

One example in index 10.
	Processed Frame:
		P1 Action(one-hot) Shape:	[1, 54]
		P1 Hp(norm.):			0.7375
		P2 Hp(norm.):			0.6425
		X Dist(norm.):			0.9999999999999999
		Y Dist(norm.):			1.0
		Label Shape:			[1, 11]


### Neural Network Design

Our neural network has two hidden layers in this test. They has 12 and 8 neurons respectively.

In [9]:
# Constants
INPUT_LAYER_SIZE = tr_data.shape[1]
OUTPUT_LAYER_SIZE = tr_labels.shape[1]

In [10]:
def create_model(neuron_count):
    model = Sequential()
    
    model.add(Dense(neuron_count, input_dim=INPUT_LAYER_SIZE, activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(OUTPUT_LAYER_SIZE, activation='softmax'))
    print( OUTPUT_LAYER_SIZE )
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

In [11]:
COMPLEXITIES = [256]
EPOCH = 100
BATCH = 16

In [12]:
if not os.path.exists(OUT_FOLDER):
    os.makedirs(OUT_FOLDER)

name = "nof1"
# Create session folder.
SESSION_FOLDER = OUT_FOLDER + '/session_{}{}'.format(time.time(),name)

os.makedirs(SESSION_FOLDER)
os.makedirs(SESSION_FOLDER + '/models')
os.makedirs(SESSION_FOLDER + '/logs')
os.makedirs(SESSION_FOLDER + '/histories')

histories = []
# Create and run networks
for complexity in COMPLEXITIES:
    starting_time = time.clock()
    file_name = str(complexity)
    model = create_model(complexity)
    print('Model created with {} neurons.'.format(complexity))
    
    # Logging for tensorboard
    tensorboard = TensorBoard(log_dir= SESSION_FOLDER + "/logs/log_{}".format(file_name))
    plot_model(model, to_file='model.png', show_shapes=True)
    print('\tTraining started.')
    checkpoint = ModelCheckpoint(filepath="model.h5", save_best_only=True, verbose=0)
    history = model.fit(tr_data, tr_labels, validation_split=0.2, epochs=EPOCH,
                    shuffle=True, batch_size=BATCH, callbacks=[tensorboard,checkpoint])
    print('\tTraining ended.')
    
    model.load_weights("model.h5")
    time_taken = time.clock() - starting_time * 1000.0
    
    model_info = {
        'info': 'One hidden layer ANN with {} neurons'.format(complexity),
        'epoch': EPOCH,
        'batch': BATCH,
        'time_taken': time_taken
    }
    
    # Save model and history
    model.save(SESSION_FOLDER + '/models/model_{}.h5'.format(file_name))
    write_file(SESSION_FOLDER + '/models/config_{}.json'.format(file_name), model.to_json())
    write_file(SESSION_FOLDER + '/models/info_{}.json'.format(file_name), json.dumps(model_info))
    model.save(OUT_FOLDER + '/model.h5'.format(file_name))
    write_file(OUT_FOLDER + '/config.json'.format(file_name), model.to_json())
    write_file(OUT_FOLDER + '/info.json'.format(file_name), json.dumps(model_info))
    
    # Display result
    scores = model.evaluate(te_data, te_labels)
    print("\t\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    pickle.dump(history, open(SESSION_FOLDER + '/histories/history_{}.save'.format(file_name), 'wb'))
    
    histories.append(history)
    
    
print('Session completed successfully. {} networks created and their data saved.'.format(len(COMPLEXITIES)))

11
Model created with 256 neurons.
	Training started.
Train on 2446 samples, validate on 612 samples
Epoch 1/100
2446/2446 [==============================] - 1s 253us/step - loss: 0.8175 - acc: 0.7796 - val_loss: 0.4894 - val_acc: 0.8284
Epoch 2/100
2446/2446 [==============================] - 0s 192us/step - loss: 0.5318 - acc: 0.8050 - val_loss: 0.4474 - val_acc: 0.8399
Epoch 3/100
2446/2446 [==============================] - 0s 165us/step - loss: 0.5060 - acc: 0.8148 - val_loss: 0.4608 - val_acc: 0.8448
Epoch 4/100
2446/2446 [==============================] - 0s 199us/step - loss: 0.4908 - acc: 0.8201 - val_loss: 0.4320 - val_acc: 0.8448
Epoch 5/100
2446/2446 [==============================] - 0s 130us/step - loss: 0.4791 - acc: 0.8189 - val_loss: 0.4598 - val_acc: 0.8415
Epoch 6/100
2446/2446 [==============================] - 0s 138us/step - loss: 0.4796 - acc: 0.8164 - val_loss: 0.4355 - val_acc: 0.8448
Epoch 7/100
2446/2446 [==============================] - 0s 174us/step - loss

Epoch 60/100
2446/2446 [==============================] - 0s 152us/step - loss: 0.3768 - acc: 0.8516 - val_loss: 0.4320 - val_acc: 0.8840
Epoch 61/100
2446/2446 [==============================] - 0s 163us/step - loss: 0.3640 - acc: 0.8618 - val_loss: 0.4336 - val_acc: 0.8824
Epoch 62/100
2446/2446 [==============================] - 0s 174us/step - loss: 0.3574 - acc: 0.8659 - val_loss: 0.4541 - val_acc: 0.8693
Epoch 63/100
2446/2446 [==============================] - 0s 171us/step - loss: 0.3557 - acc: 0.8724 - val_loss: 0.4251 - val_acc: 0.8824
Epoch 64/100
2446/2446 [==============================] - 0s 172us/step - loss: 0.3576 - acc: 0.8659 - val_loss: 0.4342 - val_acc: 0.8824
Epoch 65/100
2446/2446 [==============================] - 0s 174us/step - loss: 0.3418 - acc: 0.8790 - val_loss: 0.4599 - val_acc: 0.8742
Epoch 66/100
2446/2446 [==============================] - 0s 160us/step - loss: 0.3370 - acc: 0.8790 - val_loss: 0.4506 - val_acc: 0.8873
Epoch 67/100
2446/2446 [==========

In [13]:

# Display result
scores = model.evaluate(te_data, te_labels)
print("\t\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
print(scores)


974/974 [==============================] - 0s 27us/step
	
acc: 84.50%
[0.5181070648791609, 0.8449691991786448]


In [14]:
# te_data2, te_labels2 = process_data(testing_data2)
# scores = model.evaluate(te_data2, te_labels2)
# print("\t\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
# print(scores)


NameError: name 'testing_data2' is not defined